In this notebook I will figure out with zones are in which superdistricts.

I will merge this information with both the commute flow data and the rental data.

Output:
    -One file with CL rentals that includes code for zone and superdistrict
    -One file with two columns - zone and super district for matching with commute data

In [1]:
import pandas as pd, geopandas as gpd, matplotlib.pyplot as plt, matplotlib.cm as cm, numpy as np
from matplotlib.collections import PatchCollection
from descartes import PolygonPatch
from shapely.geometry import MultiPolygon, Polygon, Point
%matplotlib inline

In [2]:
# Read in shapefiles for zones and super districts

taz = gpd.read_file('Data/SuperShp/bayarea_superdistricts.shp')
taz_zones = gpd.read_file('Data/OriginalZoneShp/Communities_of_Concern_TAZ.shp')

In [3]:
print(taz_zones.head())
# Column of interest is taz_key

   CoCFlag      GEO_ID2                                GlobalID  LowInc_30  \
0        0  06075012100  {65A7FBA7-B655-478C-9248-0F3582B80770}          1   
1        1  06075012300  {6B582A52-B64F-4716-8F58-96AB99560124}          1   
2        1  06075012500  {FA3728FD-AB54-4546-896E-CBEC4B8FF83A}          1   
3        1  06075012400  {F690C928-C472-4240-B0BF-44B778FF99B6}          1   
4        1  06075011500  {CEAEEC36-02C5-4B18-BEC3-97A7975FDAF3}          1   

   Minority_7  OBJECTID     ShapeSTAre   ShapeSTLen  \
0           0         1  198230.878906  2019.762990   
1           0         2  296337.117188  2278.770513   
2           0         3  319910.906250  2490.141633   
3           0         4  726428.460938  3616.212017   
4           1         5  255033.156250  2019.084938   

                                            geometry  taz_key  
0  POLYGON ((-122.4133839994479 37.78770299971768...        6  
1  POLYGON ((-122.4128190004567 37.78488599965646...        7  
2  POLYG

In [4]:
len(taz_zones)

1454

In [5]:
len(taz)

34

In [6]:
print(taz.head())
# Note: column of interest is SUPERD

    AREALAND   AREAWATER  COUNT  COUNTY   LANDACRE  SUPERD  TOTPOP2000  \
0   21351865   499231994     46       1   5276.161       1      134389   
1   33875058   127195091     49       1   8370.710       2      219626   
2   51078050     4718589     78       1  12621.662       3      312465   
3   78751659  3231042006     34       1  19459.960       4      148678   
4  149481962   240621625     58       2  36937.799       5      287437   

    WATERACRE                                           geometry  
0  123362.918  (POLYGON ((550895.3255083872 4186570.307938436...  
1   31430.593  (POLYGON ((542551.6688444372 4181508.071199737...  
2    1165.989  (POLYGON ((554542.5729141268 4179392.958145962...  
3  798407.901  (POLYGON ((499731.850956357 4172984.909835728,...  
4   59458.901  POLYGON ((552046.2235804738 4160293.877120561,...  


Next couple cells makes sure that the projection is right so they can be merged spatially

In [7]:
taz.crs

{'init': 'epsg:26910'}

In [8]:
taz_zones.crs

{'init': 'epsg:4326'}

In [9]:
rentals = pd.read_csv('Data/rents_indexed.csv')
geometry = [Point(xy) for xy in zip(rentals.longitude, rentals.latitude)]
geo_rentals = gpd.GeoDataFrame(rentals, geometry=geometry)

In [10]:
# Label CRS to lat/lon 

original_crs = {'init':'epsg:4326'}
geo_rentals.crs = original_crs

In [11]:
print(geo_rentals.crs)
print(taz_zones.crs)
print(taz.crs)

{'init': 'epsg:4326'}
{'init': 'epsg:4326'}
{'init': 'epsg:26910'}


In [12]:
geo_rentals = geo_rentals.to_crs(taz.crs)
taz_zones = taz_zones.to_crs(taz.crs)

In [13]:
print(geo_rentals.crs)
print(taz_zones.crs)
print(taz.crs)

{'init': 'epsg:26910'}
{'init': 'epsg:26910'}
{'init': 'epsg:26910'}


Next I use a spatial merge to join the rental data with the zones, so that each rental listing also has a zone listed. Will use this with the commute data later.

#taz_rentals = gpd.sjoin(geo_rentals, taz, how='left', op='within')

 Note: For reasons unkown to me, the first sjoin of a notebook works fine, but when I 
 try a second, it takes forever to run and never completes.

 For this reason, instead of joining rentals with the zones, and then that 
 dataframe with superdistricts, I am first joining the zones with rentals.
 
 After that I find the centroid of the zones and join those centroids with the sueprdistricts.
 This goes quite fast, and avoids another problem - if you intersect the zones with the superdistricts some that
 seem to be touching the edges aren't counted as in the superdistrict (this is a working hypothesis, not sure what exactly happened but I many of the zones had NaNs for superdistricts when another method was used). 

In [14]:


taz_rentals2 = gpd.sjoin(geo_rentals, taz_zones, how='left', op='within')

In [15]:
taz_rentals2.head()

,neighborhood,title,rent,bedrooms,pid,date,link,sqft,sourcepage,longitude,...,index_right,CoCFlag,GEO_ID2,GlobalID,LowInc_30,Minority_7,OBJECTID,ShapeSTAre,ShapeSTLen,taz_key
0,oakland north / temescal,2 Brm+ 1 study + 2Ba in enclosed property - To...,2150.0,3.0,4.528973e+09,2014-06-19,http://sfbay.craigslist.org/eby/apa/4528973466...,NaN,http://sfbay.craigslist.org/apa/index1200.html,-122.254525,...,972.0,0.0,06001401200,{7BA880B1-802C-4987-8D07-15E503DE843F},0.0,0.0,973.0,1.065169e+06,5052.595805,975.0
1,noe valley,Attractive 2 bed in the heart of Noe Valley wi...,3200.0,2.0,4.493998e+09,2014-05-29,http://sfbay.craigslist.org/sfc/apa/4493998171...,NaN,http://sfbay.craigslist.org/apa/index3200.html,-122.427200,...,77.0,0.0,06075021400,{6AC4217B-FC70-4D79-848B-62F77A7AC89A},0.0,0.0,78.0,5.589527e+05,3105.877433,97.0
2,pacific heights,Charming Pacific Heights Edwardian steps to Fi...,1500.0,NaN,4.492178e+09,2014-05-28,http://sfbay.craigslist.org/sfc/apa/4492178358...,NaN,http://sfbay.craigslist.org/apa/index3200.html,-122.437821,...,28.0,0.0,06075013400,{AF5E0F96-CCC0-4DC9-B62F-3CDC32BC7BD6},0.0,0.0,29.0,5.683611e+05,3256.613653,48.0
3,mountain view,Charming home in Downtown area. Blocks to Cast...,2995.0,2.0,4.466669e+09,2014-05-15,http://sfbay.craigslist.org/pen/apa/4466668810...,850.0,http://sfbay.craigslist.org/apa/index2900.html,-122.079902,...,372.0,0.0,06085509600,{0AEC9F4F-9FB4-4C9D-AA73-F8AC76C55FAA},0.0,0.0,373.0,1.624917e+06,6363.483918,378.0
4,mill valley,Sunny Renovated Home With Hot Tub Level Yard ...,7500.0,4.0,4.553095e+09,2014-07-06,http://sfbay.craigslist.org/nby/apa/4553095464...,2400.0,http://sfbay.craigslist.org/apa/index500.html,-122.557780,...,1450.0,0.0,06041128200,{750BFCAA-EEDA-4C68-8C1D-ACA99D09F10B},0.0,0.0,1451.0,6.440696e+06,22855.929170,1451.0


In [16]:
len(taz_rentals2)

139824

Initial code I wrote:

zones_and_districts = gpd.sjoin(taz_zones, taz, how='left', op='within')

This seemed to work, but many of the zones did not recieve super districts:

len(zones_and_districts['SUPERD'].dropna()) = 940
(instead of 1454, which it should be)

I am assuing here it is not working because of the method to intersecting. The polygons that lie on 
the edge of a super district are not entirely within that superdistrict, so it won't record it.

To account for this, I calculated the centroids of the zones and then inersected those with the super districts,
using within.

Note: I am worried about an alternative possibility - that the zones from the MTC actually cross the borders of the super districts for the NYU depository. Not sure how to check this without plotting, and plotting in python is a challenge. Will try to load them onto carto to get a better idea. 

In [17]:
# Try to find centoirds of zones, then intersect with taz super districts

# Make a new dataframe same as taz_zones but change the geometry to centroid points
taz_centroids = taz_zones
taz_centroids['geometry'] = taz_zones['geometry'].centroid

In [18]:
zones_and_districts = gpd.sjoin(taz_centroids, taz, how='left', op='within')

In [19]:
zones_and_districts[:4]

,CoCFlag,GEO_ID2,GlobalID,LowInc_30,Minority_7,OBJECTID,ShapeSTAre,ShapeSTLen,geometry,taz_key,index_right,AREALAND,AREAWATER,COUNT,COUNTY,LANDACRE,SUPERD,TOTPOP2000,WATERACRE
0,0,06075012100,{65A7FBA7-B655-478C-9248-0F3582B80770},1,0,1,198230.878906,2019.762990,POINT (551779.926430462 4182548.765352709),6,0.0,21351865.0,499231994.0,46.0,1.0,5276.161,1.0,134389.0,123362.918
1,1,06075012300,{6B582A52-B64F-4716-8F58-96AB99560124},1,0,2,296337.117188,2278.770513,POINT (551821.8933959281 4182290.042572366),7,0.0,21351865.0,499231994.0,46.0,1.0,5276.161,1.0,134389.0,123362.918
2,1,06075012500,{FA3728FD-AB54-4546-896E-CBEC4B8FF83A},1,0,3,319910.906250,2490.141633,POINT (551822.7386776448 4181937.042687816),8,0.0,21351865.0,499231994.0,46.0,1.0,5276.161,1.0,134389.0,123362.918
3,1,06075012400,{F690C928-C472-4240-B0BF-44B778FF99B6},1,0,4,726428.460938,3616.212017,POINT (551306.0527949079 4181647.324361625),9,0.0,21351865.0,499231994.0,46.0,1.0,5276.161,1.0,134389.0,123362.918


In [20]:
# we don't need all those columns - only keep useful ones
useful_cols = ['taz_key', 'SUPERD']
z_d_subset = zones_and_districts[useful_cols]

In [21]:
z_d_subset[:3]

,taz_key,SUPERD
0,6,1.0
1,7,1.0
2,8,1.0


In [22]:
z_d_subset.to_csv('Data/zone_district_key.csv')

In [23]:
# Megrge the rental data, which has taz zones, with the zones and districts
# which has super district ids

# Select useful columns later

rentals_complete = pd.merge(taz_rentals2, zones_and_districts, on='taz_key', how='left')

In [24]:
len(rentals_complete)

139824

In [25]:
# All rental listings, with far more columns than needed. Can subset later.
rentals_complete.to_csv('Data/rentals_with_tazIDs.csv')